In [3]:
import pandas as pd

# Загрузим просто чтобы посмотреть
transactions = pd.read_excel('customer_and_transaction.xlsx', sheet_name='transaction')
customers = pd.read_excel('customer_and_transaction.xlsx', sheet_name='customer')

In [5]:
display(transactions.head())
display(customers.head())

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost
0,1,2,2950,2017-02-25,False,Approved,Solex,Standard,medium,medium,71.49,53.62
1,2,3,3120,2017-05-21,True,Approved,Trek Bicycles,Standard,medium,large,2091.47,388.92
2,3,37,402,2017-10-16,False,Approved,OHM Cycles,Standard,low,medium,1793.43,248.82
3,4,88,3135,2017-08-31,False,Approved,Norco Bicycles,Standard,medium,medium,1198.46,381.10
4,5,78,787,2017-10-01,True,Approved,Giant Bicycles,Standard,medium,large,1765.30,709.48


,customer_id,first_name,last_name,gender,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,address,postcode,state,country,property_valuation
0,1,Laraine,Medendorp,F,1953-10-12 00:00:00,Executive Secretary,Health,Mass Customer,N,Yes,060 Morning Avenue,2016,New South Wales,Australia,10
1,2,Eli,Bockman,Male,1980-12-16 00:00:00,Administrative Officer,Financial Services,Mass Customer,N,Yes,6 Meadow Vale Court,2153,New South Wales,Australia,10
2,3,Arlin,Dearle,Male,1954-01-20 00:00:00,Recruiting Manager,Property,Mass Customer,N,Yes,0 Holy Cross Court,4211,QLD,Australia,9
3,4,Talbot,NaN,Male,1961-10-03 00:00:00,NaN,IT,Mass Customer,N,No,17979 Del Mar Point,2448,New South Wales,Australia,4
4,5,Sheila-kathryn,Calton,Female,1977-05-13 00:00:00,Senior Editor,NaN,Affluent Customer,N,Yes,9 Oakridge Court,3216,VIC,Australia,9


### Нормальные формы.

- Единственное исправление для приведения данных в 1NF (первую нормальную форму), это разделение customers.address на две колонки: customers.address_number и customers.address_street. В остальном значения всех ячеек атомарны и не делимы, соответсвенно таблицы, которым их будут содержать, будут иметь первую нормальную форму.

И кстати address_number должен быть не числом, а строкой, судя по нулям перед некоторыми значениями.

- Видим, что таблица transaction изначально не соответсвует 2NF (второй нормально форме). Например значения в колонке standard_cost зависят от ключа product_id, а не от transaction_id. Поэтому надо таблицу резать на две: products и transactions

- Таблица customer не соответсвует 3NF. state и country на самом деле связаны с postcode. Есть выделить их в отдельную таблицу locations.

Итого у нас получаются таблицы со следующим набором полей:

`locations:`
- postcode
- state
- country



`products:`
- id
- brand
- product_line  
- product_class
- product_size
- list_price
- standard_cost

`transactions:`
- id
- product_id
- customer_id
- transaction_date
- online_order
- order_status

`customers`:
- id
- first_name
- last_name
- gender
- DOB
- job_title
- job_industry_category
- wealth_segment
- deceased_indicator
- owns_car
- address_number
- address_street
- postcode
- property_valuation

### Схема БД

![Компьютер](scheme.svg)

### Скрипт создания таблиц (Postgres)

In [ ]:
CREATE TABLE "locations" (
  "postcode" VARCHAR(10) PRIMARY KEY,
  "state" VARCHAR(255) NOT NULL,
  "country" VARCHAR(255) NOT NULL
)

CREATE TABLE "customers" (
  "id" SERIAL PRIMARY KEY,
  "first_name" VARCHAR(255) NOT NULL,
  "last_name" VARCHAR(255) NOT NULL,
  "gender" VARCHAR(10),
  "DOB" DATE,
  "job_title" VARCHAR(255),
  "job_industry_category" VARCHAR(255),
  "wealth_segment" VARCHAR(255),
  "deceased_indicator" CHAR(1) NOT NULL,
  "owns_car" BOOLEAN,
  "address_number" VARCHAR(10),
  "address_street" VARCHAR(255),
  "postcode" VARCHAR(10) REFERENCES locations(postcode),
  "property_valuation" INTEGER
);

CREATE TABLE "products" (
  "id" SERIAL PRIMARY KEY,
  "brand" VARCHAR(255),
  "product_line" VARCHAR(255),
  "product_class" VARCHAR(255),
  "product_size" VARCHAR(255),
  "list_price" DECIMAL(10,2),
  "standard_cost" DECIMAL(10,2)
);

CREATE TABLE "transactions" (
  "id" SERIAL PRIMARY KEY,
  "product_id" SERIAL,
  "customer_id" SERIAL,
  "transaction_date" DATE,
  "online_order" BOOLEAN,
  "order_status" VARCHAR(255)
);

ALTER TABLE "transactions" ADD FOREIGN KEY ("product_id") REFERENCES "products" ("id");

ALTER TABLE "transactions" ADD FOREIGN KEY ("customer_id") REFERENCES "customers" ("id");

### Диаграмма созданных таблицы в Postgres с помощью DBeaver

![Компьютер](diagramm.png)

Итого у нас 4 таблицы с 1NF и 3NF. 2NF не применима, так как везде простой первичный ключ

### Мелкие исправления.

1. В колонке state значение "NSW", это тоже самое, что "New South Wales". Заменяю руками, иначе потом будет конфликт.
2. Колонка owns_car Yes меняем на true, No на false
3. Не забыть что мы разделили колонку address. При импорте для начала отправим её в address_street, а потом разделим скриптом.

In [ ]:
UPDATE customers
SET address_number = split_part(address_street, ' ', 1), -- Извлекаем номер дома
  address_street = substring(address_street from position(' ' in address_street) + 1); -- Оставляем только улицу

4. Поле gender не нормализовано. Сначала импортируем как есть, потом исправим скриптом, потом повесим ограничение на выбор из 2 вариантов значения: Male, Female. U - это допустим не известно, NULL.

In [ ]:
update customers set gender='Female' where gender='F';
update customers set gender='Female' where gender='Femal';
update customers set gender='Male' where gender='M';
update customers set gender=null where gender='U';

select distinct gender from customers s;

ALTER TABLE customers
ADD CONSTRAINT gender_check
CHECK (gender IN ('Male', 'Female') OR gender IS NULL);

5. Содержимое колонки transaction_date сначала импортируем в во временную колонку temp_date:varchar. Мы ж не можем студентам простое ДЗ дать, нам надо чтобы они страдали. Поэтому формат даты у нас 'DD.MM.YYYY', мы его потом конвертнём следующим скриптом:

In [ ]:
UPDATE transactions
SET transaction_date = to_date(temp_date, 'DD.MM.YYYY')
WHERE temp_date IS NOT NULL

6. Поле customer_id у transactions, разрешаем null. Отмечаем пустыми идентификаторы customer_id: 577 и 5034. Таких идентификаторов в customers у нас нет.

### Далее импорт двух csv в 4 таблицы срикншотами.

Сохраняем исходный customer_and_transaction.xlsx в две таблицы customer.csv и transaction.csv, кликаем правой кнопкой на public и начинаем импорт

#### Настройки импорта для locations
![import 1. locations.png](./images/import%201.%20locations.png)

#### Данные в locations
![result 1. locations.png](./images/result%201.%20locations.png)

#### Настройки импорта для customers
![import 2. customers.png](./images/import%202.%20customers.png)

#### Данные в customers
![result 2. customers.png](./images/result%202.%20customers.png)

#### Настройки импорта для products
![import 3. products.png](./images/import%203.%20products.png)

#### Данные в products
![result 3. products.png](./images/result%203.%20products.png)

#### Настройки импорта для transactions
![import 4. products.png](./images/import%204.%20transactions.png)

#### Данные в transactions
![result 4. transactions.png](./images/result%204.%20transactions.png)